# Tanscripting Video Files From TikTok

#### Installing dependencies
We need moviepy to convert the videos to mp3
We need openai-whisper to tanscript the files to text

In [ ]:
%pip install moviepy
%pip install -U openai-whisper
%pip install faster-whisper 
%pip install pandas 
%pip install requests
%pip install moviepy
%pip install -U spacy

Set your notebook to the right directrory

In [ ]:

%cd '/Users/noursafadi/Documents/Uni/Parsons-Spring-25/Major Studio 02/Thesis/tiktok-scraper'

Importing

In [ ]:
from moviepy import VideoFileClip
from moviepy import *
import whisper
import csv
import requests
from faster_whisper import WhisperModel
import httpx
import json
from os import listdir, path
import pandas as pd
import spacy

Downloading video files

In [ ]:
with open("tiktok_vids.json", "r") as file: 
    data = json.load(file)

links = []
for idx,post in enumerate(data):
    if "videoMeta" in post and "downloadAddr" in post["videoMeta"]:
        url = post["videoMeta"]["downloadAddr"]
        if url: 
            video_response = httpx.get(url)
            file_path = f"/videos/{idx}.mp4"
            with open(file_path, "wb") as f:
                f.write(video_response.content)
            print(f"{idx}: Seccessful")
        else:
            print(f"{idx} is not Available")
    else:
        print(f"{idx} is Not Available")

Converting from video to audio

In [ ]:
video_path = "/videos"
audio_output_path = "/audio"

files = [f for f in listdir(video_path) if f.endswith(".mp4")]

for fname in files:
    video_location = path.join(video_path, fname)

    try: 
        video = VideoFileClip(video_location)
        audio = video.audio

        audio_file_name = path.splitext(fname)[0] + ".mp3"
        audio_file_path = path.join(audio_output_path, audio_file_name)
        audio.write_audiofile(audio_file_path, codec="mp3")

        video.close()
    except Exception as e: 
        print(f"Error Processing {fname} : {e}")

Lodaing Whisper object Model from Open AI


In [ ]:
#OpenAI WHISPER basic transcripting
model = whisper.load_model("turbo")

Transcripting Audio Files with Whisper AI 

In [ ]:
audio_folder = "audio/"
audio_files = [f for f in listdir(audio_folder) if f.endswith(".mp3")]

scripts = []
for fname in audio_files:
    file = path.join(audio_folder, fname)
    result = model.transcribe(file)
    scripts.append(result["text"])

Creating a JSON file 

In [ ]:
#note when using json with a different language, letters such as ä, ö or simillar will be encoded so we will need to deal with that. 
# That's why I will create a csv file, which keep the letters as they are without any change. 
json_sting = json.dumps(scripts)
json_path = "data/scripts.json"
with open(json_path, "w") as file: 
    json.dump(scripts,file,indent=4)

Create a CSV file

In [ ]:
csv_path = "data/scripts.csv"

# Open CSV File for Writing
with open(csv_path, mode="w", newline="", encoding="utf-8") as file:
    writer = csv.writer(file)

    # Write Header
    writer.writerow(["id", "script"])

    # Process Each Transcribed Segment
    for idx, text in enumerate(scripts):
        writer.writerow([idx, text])

print(f"Transcription and translation saved to {csv_path}")

In [ ]:
#reading csv as data frame. 
df = pd.read_csv("data/scripts.csv")

Translating the texts with DEEPL API

In [ ]:
#DEEPL API KEY
DEEPL_API_KEY = "39f5e8f7-4926-44de-b567-420c316bc88e:fx"  # Replace with your actual key

# Define DeepL Translation Function
def translate_text(text, source_lang="DE", target_lang="EN"):
    url = "https://api-free.deepl.com/v2/translate"  # Use "api.deepl.com" for Pro accounts
    params = {
        "auth_key": DEEPL_API_KEY,
        "text": text,
        "source_lang": source_lang,
        "target_lang": target_lang
    }
    response = requests.post(url, data=params)
    if response.status_code == 200:
        return response.json()["translations"][0]["text"]
    else:
        print("Translation Error:", response.text)
        return text  # Return original text in case of failure

In [ ]:
translated_list = []
for index, row in df.iterrows():
    #print(f"{row['script']}")
    translated_list.append(translate_text(row['script']))
    

In [ ]:
csv_path = "data/translated_scripts.csv"

# Open CSV File for Writing
with open(csv_path, mode="w", newline="", encoding="utf-8") as file:
    writer = csv.writer(file)

    # Write Header
    writer.writerow(["id", "script"])

    # Process Each Transcribed Segment
    for idx, text in enumerate(translated_list):
        writer.writerow([idx, text])

print(f"Transcription and translation saved to {csv_path}")

Spacy Frame work to handel categories and keyworsd

In [104]:
# Load English tokenizer, tagger, parser and NER
nlp = spacy.load("en_core_web_sm")
    
# Process whole documents
text = ("When Sebastian Thrun started working on self-driving cars at "
        "Google in 2007, few people outside of the company took him "
        "seriously. “I can tell you very senior CEOs of major American "
        "car companies would shake my hand and turn away because I wasn’t "
        "worth talking to,” said Thrun, in an interview with Recode earlier "
        "this week.")
doc = nlp(text)

# Analyze syntax
print("Noun phrases:", [chunk.text for chunk in doc.noun_chunks])
print("Verbs:", [token.lemma_ for token in doc if token.pos_ == "VERB"])

# Find named entities, phrases and concepts
for entity in doc.ents:
    print(entity.text, entity.label_)


Noun phrases: ['Sebastian Thrun', 'self-driving cars', 'Google', 'few people', 'the company', 'him', 'I', 'you', 'very senior CEOs', 'major American car companies', 'my hand', 'I', 'Thrun', 'an interview', 'Recode']
Verbs: ['start', 'work', 'drive', 'take', 'tell', 'shake', 'turn', 'talk', 'say']
Sebastian Thrun PERSON
Google ORG
2007 DATE
American NORP
Thrun GPE
Recode ORG
earlier this week DATE


In [ ]:
df_translated = pd.read_csv("data/translated_scripts.csv")
nlp = spacy.load("en_core_web_sm")

for index, row in df_translated.iterrows():
    print(index)
    #print(row["script"])
    text = row["script"]
    doc = nlp(text)
    for entity in doc.ents:
            print("This is an Entity",entity.text, entity.label_)
    print("Noun phrases:", [chunk.text for chunk in doc.noun_chunks])
    print("Verbs: ", [token.lemma_ for token in doc if token.pos_ =="VERB"])

0
This is an Entity AfD PERSON
This is an Entity Germany GPE
This is an Entity February 23 DATE
This is an Entity Germany GPE
Noun phrases: [' A country', 'engineers', 'inventors', 'bans', 'No combustion engines', 'no oil and gas heating', 'We', 'no more ideological experiments', 'The AfD', 'genuine technological openness', 'paternalism', 'Progress', 'a diversity', 'ideas', 'green regulations', 'bans', 'We', 'Germany', 'a country', 'innovation', 'February', 'Alternative', 'Germany']
Verbs:  ['slow', 'say', 'commit', 'research', 'come', 'turn', 'vote']
1
This is an Entity the hundreds of thousands CARDINAL
This is an Entity AfD PERSON
This is an Entity February 23 DATE
Noun phrases: [' The completely uncontrolled, illegal, mass immigration', 'People', 'our country', 'the hundreds', 'thousands', 'We', 'that', 'That', 'we', 'AfD', 'February']
Verbs:  ['uncontrolle', 'pour', 'cope', 'vote']
2
This is an Entity Friedrich Merz PERSON
This is an Entity December DATE
This is an Entity one CARD